#### RGB vs BGR

Зрозуміти порядок каналів для BMP-зображення можна шляхом маніпуляції пікселів і обнулення певних каналів. У BMP форматі кольорові канали найчастіше представлені у порядку BGR (синій, зелений, червоний), а не RGB. Тобто, перший канал відповідає за синій, другий — за зелений, а третій — за червоний. Тому при обробці таких зображень та розрахунку метрик, особливо після перетворення з одного формату в інший важливо враховувати що позиція каналу може відрізнятись між вхідними зображеннями та вихідними.

In [1]:
from PIL import Image
import numpy as np

In [6]:
def zero_channels(image_path, output_path, keep_channel):
    """
    Замінює два з трьох кольорових каналів RGB(PNG)-зображення на 0.
    
    :param image_path: Шлях до вихідного зображення.
    :param output_path: Шлях для збереження модифікованого зображення.
    :param keep_channel: Канал, який потрібно зберегти (0 - Червоний, 1 - Зелений, 2 - Синій).
    """
    # Відкриття зображення
    image = Image.open(image_path).convert("RGBA")  # Примусово конвертуємо зображення в RGBA (4 канали)
    
    # Перетворення зображення в дані у форматі NumPy
    image_data = np.array(image)
    
    # Виділяємо RGB-канали, ігноруючи альфа-канал (якщо він є)
    rgb_data = image_data[:, :, :3]
    alpha_channel = image_data[:, :, 3] if image_data.shape[2] == 4 else None

    # Створюємо масив для зберігання результату
    modified_data = np.zeros_like(rgb_data)
    
    # Залишаємо тільки обраний канал
    modified_data[:, :, keep_channel] = rgb_data[:, :, keep_channel]
    
    # Якщо є альфа-канал, додаємо його назад
    if alpha_channel is not None:
        # Додаємо альфа-канал назад до модифікованих даних
        modified_data_with_alpha = np.zeros_like(image_data)
        modified_data_with_alpha[:, :, :3] = modified_data
        modified_data_with_alpha[:, :, 3] = alpha_channel
        modified_image = Image.fromarray(modified_data_with_alpha, "RGBA")
    else:
        # Якщо альфа-каналу немає, робимо тільки RGB
        modified_image = Image.fromarray(modified_data, "RGB")
    
    # Збереження модифікованого зображення
    modified_image.save(output_path)
    print(f"Модифіковане зображення збережено за адресою: {output_path}")

In [20]:
def compare_channel_with_grayscale(bmp_image_path, grayscale_image_path, channel):
    """
    Порівнює заданий канал (R/G/B) з grayscale-зображенням, отриманим із цього каналу.
    
    :param bmp_image_path: Шлях до BMP-зображення з одним або кількома ненульовими каналами.
    :param grayscale_image_path: Шлях до grayscale-зображення.
    :param channel: Канал для перевірки (0 - Red, 1 - Green, 2 - Blue).
    :return: Опис результатів порівняння.
    """
    channel_names = ["Red", "Green", "Blue"]  # Для виведення імен каналів
    
    # Відкриття обох зображень
    bmp_image = Image.open(bmp_image_path).convert("RGB")  # Переконуємося, що це RGB
    grayscale_image = Image.open(grayscale_image_path).convert("L")  # Переконуємося, що це Grayscale (L)
    
    # Перетворення зображень у NumPy-матриці
    bmp_array = np.array(bmp_image)
    grayscale_array = np.array(grayscale_image)
    
    # Виділяємо обраний канал із BMP-зображення
    selected_channel = bmp_array[:, :, channel]  # 0 - R, 1 - G, 2 - B
    
    # Перевірка розмірів
    if selected_channel.shape != grayscale_array.shape:
        raise ValueError(f"Розмірності зображень не збігаються: {selected_channel.shape} і {grayscale_array.shape}")
    
    # Порівнюємо піксель за пікселем
    difference = selected_channel.astype(int) - grayscale_array.astype(int)
    max_diff = np.max(np.abs(difference))  # Максимальна різниця
    mean_diff = np.mean(np.abs(difference))  # Середня різниця
    
    # Результати порівняння
    if np.array_equal(selected_channel, grayscale_array):
        print(f"{channel_names[channel]} канал і grayscale-зображення є ідентичними.")
    else:
        print(f"{channel_names[channel]} канал і grayscale-зображення не ідентичні.")
        print(f"Максимальна різниця: {max_diff}")
        print(f"Середня різниця: {mean_diff}")
        print(f"Кількість пікселів із розбіжностями: {np.sum(difference != 0)}")
        coords = np.argwhere(difference != 0)  # Координати пікселів із розбіжностями
        print(f"Приклад координат із розбіжностями: {coords[:10]} (показано до 10)")


In [23]:
# Використання функції
image_path = "validate_channels/reconstructed_q1_sigma_25_img_1.png"  # Шлях до вашого BMP зображення
output_path_red = "validate_channels/reconstracted_color/red_channel.bmp"  # Зберегти тільки червоний канал
output_path_green = "validate_channels/reconstracted_color/green_channel.bmp"  # Зберегти тільки зелений канал
output_path_blue = "validate_channels/reconstracted_color/blue_channel.bmp"  # Зберегти тільки синій канал

# Виклик функції для кожного каналу
zero_channels(image_path, output_path_red, keep_channel=0)  # Тільки червоний канал
zero_channels(image_path, output_path_green, keep_channel=1)  # Тільки зелений канал
zero_channels(image_path, output_path_blue, keep_channel=2)  # Тільки синій канал

Модифіковане зображення збережено за адресою: validate_channels/reconstracted_color/red_channel.bmp
Модифіковане зображення збережено за адресою: validate_channels/reconstracted_color/green_channel.bmp
Модифіковане зображення збережено за адресою: validate_channels/reconstracted_color/blue_channel.bmp


In [9]:
# Використання функції
image_path = "validate_channels/SS1.png"  # Шлях до вашого BMP зображення
output_path_red = "validate_channels/red_channel.png"  # Зберегти тільки червоний канал
output_path_green = "validate_channels/green_channel.png"  # Зберегти тільки зелений канал
output_path_blue = "validate_channels/blue_channel.png"  # Зберегти тільки синій канал

# Виклик функції для кожного каналу
zero_channels(image_path, output_path_red, keep_channel=0)  # Тільки червоний канал
zero_channels(image_path, output_path_green, keep_channel=1)  # Тільки зелений канал
zero_channels(image_path, output_path_blue, keep_channel=2)  # Тільки синій канал

Модифіковане зображення збережено за адресою: validate_channels/red_channel.png
Модифіковане зображення збережено за адресою: validate_channels/green_channel.png
Модифіковане зображення збережено за адресою: validate_channels/blue_channel.png


In [22]:
bmp_image_path = "validate_channels/red_channel.png"
grayscale_image_path_r = "validate_channels/convert_red_channel.png"  # Файл із командою `-channel R -separate`
compare_channel_with_grayscale(bmp_image_path, grayscale_image_path_r, channel=0)  # 0 - R

bmp_image_path = "validate_channels/green_channel.png"
grayscale_image_path_g = "validate_channels/convert_green_channel.png"  # Файл із командою `-channel G -separate`
compare_channel_with_grayscale(bmp_image_path, grayscale_image_path_g, channel=1)  # 1 - G

bmp_image_path = "validate_channels/blue_channel.png"
grayscale_image_path_b = "validate_channels/convert_blue_channel.png"  # Файл із командою `-channel B -separate`
compare_channel_with_grayscale(bmp_image_path, grayscale_image_path_b, channel=2)  # 2 - B

Red канал і grayscale-зображення є ідентичними.
Green канал і grayscale-зображення є ідентичними.
Blue канал і grayscale-зображення є ідентичними.
